In [ ]:
# =====================================
# ULTIMATE POLLUTION PREDICTION MODEL
# Based on Research Guide: 25+ Academic Papers Analysis
# Expected Improvement: 25-55% RMSE reduction (from 0.5694 to 0.35-0.40)
# =====================================

import pandas as pd
import numpy as np
import warnings
import os
import json
import joblib
from datetime import datetime
from scipy import stats
from scipy.stats import boxcox, yeojohnson
import gc

# Core ML Libraries
from sklearn.model_selection import (
    train_test_split, cross_val_score, TimeSeriesSplit, KFold,
    StratifiedKFold, GroupKFold
)
from sklearn.preprocessing import (
    RobustScaler, StandardScaler, MinMaxScaler, PowerTransformer,
    PolynomialFeatures, QuantileTransformer
)
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    explained_variance_score, median_absolute_error
)
from sklearn.feature_selection import (
    SelectKBest, f_regression, RFE, SelectFromModel,
    mutual_info_regression
)
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Advanced Models
from sklearn.ensemble import (
    RandomForestRegressor, ExtraTreesRegressor,
    VotingRegressor, StackingRegressor, BaggingRegressor
)
from sklearn.linear_model import (
    Ridge, Lasso, ElasticNet, HuberRegressor,
    QuantileRegressor, TheilSenRegressor, LinearRegression
)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor

# Gradient Boosting Libraries
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

# Hyperparameter Optimization
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

# Advanced Analysis (if available)
try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    SHAP_AVAILABLE = False
    print("SHAP not available - feature importance analysis will be limited")

warnings.filterwarnings('ignore')
np.random.seed(42)

# Create directories
directories = [
    'models', 'models/individual', 'models/ensembles', 'models/robust',
    'results', 'submissions', 'feature_analysis', 'transformations'
]
for directory in directories:
    os.makedirs(directory, exist_ok=True)

print("🚀 ULTIMATE POLLUTION PREDICTION MODEL")
print("📚 Based on 25+ Academic Papers Research")
print("🎯 Target: 25-55% RMSE Improvement")
print("=" * 80)

# =====================================
# ADVANCED FEATURE ENGINEERING V3 (PROVEN TECHNIQUES)
# =====================================

def create_ultimate_features(df, is_train=True, target_encodings=None):
    """
    Ultimate feature engineering incorporating proven techniques from previous phases
    Combined with research-backed innovations
    """
    df_ultimate = df.copy()
    print(f"🔧 Creating ultimate features... Initial shape: {df_ultimate.shape}")
    
    # Preserve target column if it exists (for training data)
    target_col = None
    if 'target' in df_ultimate.columns:
        target_col = df_ultimate['target'].copy()
        print("Target column preserved for feature engineering")
    elif 'pollution_value' in df_ultimate.columns:
        target_col = df_ultimate['pollution_value'].copy()
        print("Pollution_value target column preserved for feature engineering")
    
    # 1. ADVANCED MISSING VALUE HANDLING (Enhanced from previous work)
    print("📊 Advanced missing value imputation...")
    initial_nan_count = df_ultimate.isnull().sum().sum()
    if initial_nan_count > 0:
        print(f"Found {initial_nan_count} NaN values, filling with appropriate values...")
        
        # Fill numerical columns with median (excluding target)
        numerical_cols = df_ultimate.select_dtypes(include=[np.number]).columns
        numerical_cols = [col for col in numerical_cols if col not in ['target', 'pollution_value', 'id']]
        for col in numerical_cols:
            if df_ultimate[col].isnull().any():
                df_ultimate[col].fillna(df_ultimate[col].median(), inplace=True)
        
        # Fill categorical columns with mode
        categorical_cols = df_ultimate.select_dtypes(include=['object']).columns
        for col in categorical_cols:
            if df_ultimate[col].isnull().any():
                df_ultimate[col].fillna(df_ultimate[col].mode()[0] if len(df_ultimate[col].mode()) > 0 else 'unknown', inplace=True)
    
    # 2. POLYNOMIAL INTERACTIONS of key features (Proven from Phase 5)
    key_features = ['latitude', 'longitude', 'hour']
    available_key_features = [f for f in key_features if f in df_ultimate.columns]
    
    if len(available_key_features) >= 2:
        print(f"Creating polynomial features from: {available_key_features}")
        
        # Ensure no NaN values in key features
        for feature in available_key_features:
            if df_ultimate[feature].isnull().any():
                df_ultimate[feature].fillna(df_ultimate[feature].median(), inplace=True)
        
        # Check for any infinite values
        for feature in available_key_features:
            df_ultimate[feature] = df_ultimate[feature].replace([np.inf, -np.inf], df_ultimate[feature].median())
        
        poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
        poly_features = poly.fit_transform(df_ultimate[available_key_features])
        poly_feature_names = poly.get_feature_names_out(available_key_features)
        
        for i, name in enumerate(poly_feature_names):
            if name not in available_key_features:
                df_ultimate[f'poly_{name}'] = poly_features[:, i]
        
        print(f"Added {len(poly_feature_names) - len(available_key_features)} polynomial interaction features")
    else:
        print(f"Skipping polynomial features - only {len(available_key_features)} key features available")
    
    # 3. SPATIAL CLUSTERING (Enhanced from Phase 5)
    if 'latitude' in df_ultimate.columns and 'longitude' in df_ultimate.columns:
        print("🌍 Creating spatial clustering features...")
        coords = df_ultimate[['latitude', 'longitude']].values
        
        # DBSCAN clustering (proven technique)
        dbscan = DBSCAN(eps=0.05, min_samples=20)
        clusters = dbscan.fit_predict(coords)
        df_ultimate['spatial_cluster'] = clusters
        
        # Distance to major cluster centers
        unique_clusters = np.unique(clusters[clusters != -1])
        for cluster_id in unique_clusters[:3]:
            cluster_points = coords[clusters == cluster_id]
            if len(cluster_points) > 0:
                cluster_center = np.mean(cluster_points, axis=0)
                distances = np.sqrt(np.sum((coords - cluster_center)**2, axis=1))
                df_ultimate[f'dist_cluster_{cluster_id}'] = distances
        
        # K-means clustering with multiple granularities (research-backed)
        for n_clusters in [5, 10, 15, 25]:
            kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
            df_ultimate[f'kmeans_cluster_{n_clusters}'] = kmeans.fit_predict(coords)
            
            # Distance to nearest cluster center
            centers = kmeans.cluster_centers_
            distances = np.min(np.sqrt(
                ((coords[:, np.newaxis] - centers) ** 2).sum(axis=2)
            ), axis=1)
            df_ultimate[f'distance_to_kmeans_{n_clusters}'] = distances
    
    # 4. ENHANCED CYCLICAL FEATURES (From Phase 5)
    print("⏰ Creating enhanced cyclical features...")
    time_features = {
        'hour': 24,
        'day_of_week': 7,
        'day_of_year': 365.25,
        'month': 12
    }
    
    for feature, period in time_features.items():
        if feature in df_ultimate.columns:
            # Multiple harmonics for better temporal capture
            for harmonic in [1, 2, 3]:
                df_ultimate[f'{feature}_sin_h{harmonic}'] = np.sin(2 * np.pi * harmonic * df_ultimate[feature] / period)
                df_ultimate[f'{feature}_cos_h{harmonic}'] = np.cos(2 * np.pi * harmonic * df_ultimate[feature] / period)
    
    # 5. STATISTICAL AGGREGATIONS (From Phase 5)
    if 'latitude' in df_ultimate.columns and 'longitude' in df_ultimate.columns:
        print("📊 Creating spatial binning features...")
        # Spatial binning
        df_ultimate['lat_bin'] = pd.cut(df_ultimate['latitude'], bins=15, labels=False)
        df_ultimate['lon_bin'] = pd.cut(df_ultimate['longitude'], bins=15, labels=False)
        
        # Combined spatial bin
        df_ultimate['spatial_bin'] = df_ultimate['lat_bin'] * 100 + df_ultimate['lon_bin']
    
    # 6. DISTANCE-BASED FEATURES (Enhanced from Phase 5)
    if 'latitude' in df_ultimate.columns and 'longitude' in df_ultimate.columns:
        print("📏 Creating distance-based features...")
        lat_center = df_ultimate['latitude'].mean()
        lon_center = df_ultimate['longitude'].mean()
        
        df_ultimate['distance_from_center'] = np.sqrt(
            (df_ultimate['latitude'] - lat_center)**2 + 
            (df_ultimate['longitude'] - lon_center)**2
        )
        
        # Distance from boundaries
        df_ultimate['dist_from_lat_min'] = df_ultimate['latitude'] - df_ultimate['latitude'].min()
        df_ultimate['dist_from_lat_max'] = df_ultimate['latitude'].max() - df_ultimate['latitude']
        df_ultimate['dist_from_lon_min'] = df_ultimate['longitude'] - df_ultimate['longitude'].min()
        df_ultimate['dist_from_lon_max'] = df_ultimate['longitude'].max() - df_ultimate['longitude']
        
        # Distance from pollution hotspots (research-backed)
        if is_train and target_col is not None:
            # Find pollution hotspots (95th percentile locations)
            high_pollution_mask = target_col >= target_col.quantile(0.95)
            if high_pollution_mask.sum() > 0:
                hotspot_lat = df_ultimate.loc[high_pollution_mask, 'latitude'].mean()
                hotspot_lon = df_ultimate.loc[high_pollution_mask, 'longitude'].mean()
                df_ultimate['distance_from_hotspot'] = np.sqrt(
                    (df_ultimate['latitude'] - hotspot_lat)**2 + 
                    (df_ultimate['longitude'] - hotspot_lon)**2
                )
    
    # 7. TEMPORAL PATTERNS (From Phase 5)
    if 'hour' in df_ultimate.columns:
        print("� Creating temporal pattern features...")
        # Rush hour indicators
        df_ultimate['is_morning_rush'] = ((df_ultimate['hour'] >= 7) & (df_ultimate['hour'] <= 9)).astype(int)
        df_ultimate['is_evening_rush'] = ((df_ultimate['hour'] >= 17) & (df_ultimate['hour'] <= 19)).astype(int)
        df_ultimate['is_rush_hour'] = (df_ultimate['is_morning_rush'] | df_ultimate['is_evening_rush']).astype(int)
        
        # Time of day categories
        df_ultimate['time_category'] = pd.cut(df_ultimate['hour'], 
                                            bins=[0, 6, 12, 18, 24], 
                                            labels=['night', 'morning', 'afternoon', 'evening'],
                                            include_lowest=True)
        df_ultimate['time_category_encoded'] = pd.Categorical(df_ultimate['time_category']).codes
        
        # Additional research-backed temporal features
        df_ultimate['is_night'] = (df_ultimate['hour'] >= 22) | (df_ultimate['hour'] <= 6)
        df_ultimate['is_peak_pollution'] = (df_ultimate['hour'] >= 8) & (df_ultimate['hour'] <= 18)
        df_ultimate['hour_squared'] = df_ultimate['hour'] ** 2
        df_ultimate['hour_cubed'] = df_ultimate['hour'] ** 3
    
    if 'day_of_week' in df_ultimate.columns:
        df_ultimate['is_weekend'] = (df_ultimate['day_of_week'] >= 5).astype(int)
        
        if 'hour' in df_ultimate.columns:
            df_ultimate['weekend_hour_interaction'] = df_ultimate['is_weekend'] * df_ultimate['hour']
    
    # 8. WEATHER PROXY FEATURES (From Phase 5)
    if all(col in df_ultimate.columns for col in ['latitude', 'longitude', 'day_of_year']):
        print("🌤️ Creating weather proxy features...")
        # Simple weather proxies
        df_ultimate['weather_proxy'] = (
            np.sin(2*np.pi*df_ultimate['day_of_year']/365) * df_ultimate['latitude'] + 
            np.cos(2*np.pi*df_ultimate['day_of_year']/365) * df_ultimate['longitude']
        )
        
        # Temperature proxy
        df_ultimate['temp_proxy'] = (
            20 + 15 * np.sin(2*np.pi*(df_ultimate['day_of_year']-80)/365) +
            5 * np.sin(2*np.pi*df_ultimate['hour']/24) -
            0.1 * np.abs(df_ultimate['latitude'])
        )
    
    # 9. INTERACTION FEATURES (Enhanced from Phase 5)
    print("� Creating interaction features...")
    numeric_cols = df_ultimate.select_dtypes(include=[np.number]).columns
    important_pairs = [
        ('latitude', 'longitude'),
        ('latitude', 'hour'),
        ('longitude', 'hour'),
        ('distance_from_center', 'hour')
    ]
    
    for col1, col2 in important_pairs:
        if col1 in numeric_cols and col2 in numeric_cols:
            df_ultimate[f'{col1}_{col2}_interaction'] = df_ultimate[col1] * df_ultimate[col2]
            df_ultimate[f'{col1}_{col2}_ratio'] = df_ultimate[col1] / (df_ultimate[col2] + 1e-8)
    
    # 10. STATISTICAL FEATURES (From Phase 5)
    numeric_cols_for_stats = [col for col in numeric_cols if col not in ['target', 'pollution_value', 'id']]
    if len(numeric_cols_for_stats) > 3:
        print("📈 Creating statistical aggregation features...")
        # Create some statistical aggregations across features
        top_features = numeric_cols_for_stats[:5]
        df_ultimate['feature_mean'] = df_ultimate[top_features].mean(axis=1)
        df_ultimate['feature_std'] = df_ultimate[top_features].std(axis=1)
        df_ultimate['feature_median'] = df_ultimate[top_features].median(axis=1)
        
        # Rolling statistics for key features
        for window in [3, 5]:
            if len(df_ultimate) > window:
                for col in top_features[:3]:  # Limit to avoid explosion
                    if df_ultimate[col].notna().sum() > window:
                        df_ultimate[f'{col}_rolling_mean_{window}'] = (
                            df_ultimate[col].rolling(window=window, min_periods=1).mean()
                        )
                        df_ultimate[f'{col}_rolling_std_{window}'] = (
                            df_ultimate[col].rolling(window=window, min_periods=1).std()
                        )
    
    # 11. ROBUST CATEGORICAL ENCODING
    print("🏷️ Advanced categorical encoding...")
    categorical_cols = df_ultimate.select_dtypes(include=['object', 'category']).columns
    
    for col in categorical_cols:
        if col in ['target', 'pollution_value', 'id', 'time_category']:
            continue
            
        unique_count = df_ultimate[col].nunique()
        
        if unique_count <= 10:
            # One-hot encoding for low cardinality
            dummies = pd.get_dummies(df_ultimate[col], prefix=col, drop_first=True)
            df_ultimate = pd.concat([df_ultimate, dummies], axis=1)
        else:
            # Target encoding for high cardinality (training only)
            if is_train and target_col is not None:
                target_mean = target_col.mean()
                encoding_map = df_ultimate.groupby(col)[target_col.name].mean()
                df_ultimate[f'{col}_target_encoded'] = (
                    df_ultimate[col].map(encoding_map).fillna(target_mean)
                )
            elif target_encodings and col in target_encodings:
                # Use pre-computed encodings for test data
                df_ultimate[f'{col}_target_encoded'] = (
                    df_ultimate[col].map(target_encodings[col]).fillna(0)
                )
        
        # Frequency encoding
        freq_map = df_ultimate[col].value_counts(normalize=True).to_dict()
        df_ultimate[f'{col}_frequency'] = df_ultimate[col].map(freq_map)
    
    # 12. OUTLIER RESISTANT FEATURES (Research-backed)
    print("🛡️ Creating outlier-resistant features...")
    key_numerical = [col for col in numeric_cols_for_stats[:5] if col in df_ultimate.columns]
    
    for col in key_numerical:
        # Winsorized features (cap at 5th and 95th percentiles)
        q05, q95 = df_ultimate[col].quantile([0.05, 0.95])
        df_ultimate[f'{col}_winsorized'] = df_ultimate[col].clip(lower=q05, upper=q95)
        
        # Rank features (robust to outliers)
        df_ultimate[f'{col}_rank'] = df_ultimate[col].rank(pct=True)
        
        # Binned features
        try:
            df_ultimate[f'{col}_binned'] = pd.qcut(
                df_ultimate[col], q=5, labels=False, duplicates='drop'
            )
        except:
            df_ultimate[f'{col}_binned'] = pd.cut(
                df_ultimate[col], bins=5, labels=False
            )
    
    # 13. FINAL CLEANUP (Enhanced from Phase 5)
    print("🧹 Final feature cleanup...")
    
    # Clean up categorical columns that can't be used directly
    df_ultimate = df_ultimate.drop(columns=['time_category'], errors='ignore')
    
    # Remove remaining object columns
    object_cols = df_ultimate.select_dtypes(include=['object']).columns
    if len(object_cols) > 0:
        print(f"Dropping remaining object columns: {list(object_cols)}")
        df_ultimate = df_ultimate.drop(columns=object_cols)
    
    # Comprehensive NaN and infinite value handling
    initial_nan = df_ultimate.isnull().sum().sum()
    initial_inf = np.isinf(df_ultimate.select_dtypes(include=[np.number])).sum().sum()
    
    if initial_nan > 0:
        print(f"Cleaning {initial_nan} NaN values...")
        numeric_columns = df_ultimate.select_dtypes(include=[np.number]).columns
        for col in numeric_columns:
            if df_ultimate[col].isnull().any():
                median_val = df_ultimate[col].median()
                if pd.isna(median_val):
                    df_ultimate[col].fillna(0, inplace=True)
                else:
                    df_ultimate[col].fillna(median_val, inplace=True)
        df_ultimate = df_ultimate.fillna(0)
    
    if initial_inf > 0:
        print(f"Cleaning {initial_inf} infinite values...")
        numeric_columns = df_ultimate.select_dtypes(include=[np.number]).columns
        for col in numeric_columns:
            if np.isinf(df_ultimate[col]).any():
                median_val = df_ultimate[col].replace([np.inf, -np.inf], np.nan).median()
                if pd.isna(median_val):
                    df_ultimate[col] = df_ultimate[col].replace([np.inf, -np.inf], 0)
                else:
                    df_ultimate[col] = df_ultimate[col].replace([np.inf, -np.inf], median_val)
    
    # Final verification
    final_nan = df_ultimate.isnull().sum().sum()
    final_inf = np.isinf(df_ultimate.select_dtypes(include=[np.number])).sum().sum()
    
    if final_nan > 0 or final_inf > 0:
        print(f"Warning: Still have {final_nan} NaN and {final_inf} infinite values")
        df_ultimate = df_ultimate.fillna(0)
        numeric_columns = df_ultimate.select_dtypes(include=[np.number]).columns
        df_ultimate[numeric_columns] = df_ultimate[numeric_columns].replace([np.inf, -np.inf], 0)
    
    # Restore target column if it was preserved
    if target_col is not None:
        if 'target' in df.columns:
            df_ultimate['target'] = target_col
        else:
            df_ultimate['pollution_value'] = target_col
        print("Target column restored after feature engineering")
    
    print(f"✅ Ultimate feature engineering completed!")
    print(f"📊 Final shape: {df_ultimate.shape}")
    print(f"🆕 Added {df_ultimate.shape[1] - df.shape[1]} new features")
    print(f"🔍 Data quality: {df_ultimate.isnull().sum().sum()} NaN, {np.isinf(df_ultimate.select_dtypes(include=[np.number])).sum().sum()} infinite values")
    
    return df_ultimate

# =====================================
# ADVANCED TARGET TRANSFORMATION
# =====================================

def optimize_target_transformation(y):
    """
    Test multiple transformation methods and select the best one
    """
    print("🎯 Optimizing target transformation...")
    
    transformations = {}
    
    def evaluate_transformation(name, transformed_y, lambda_param=None):
        # Normality test (Shapiro-Wilk)
        sample_size = min(5000, len(transformed_y))
        sample_data = transformed_y[:sample_size] if len(transformed_y) > sample_size else transformed_y
        
        try:
            _, p_value = stats.shapiro(sample_data)
        except:
            p_value = 0
        
        # Skewness and kurtosis
        skew = abs(stats.skew(transformed_y))
        kurt = abs(stats.kurtosis(transformed_y))
        
        # Combined score (higher is better)
        score = p_value + 1/(1 + skew) + 1/(1 + kurt)
        
        return {
            'name': name,
            'score': score,
            'normality_p': p_value,
            'skewness': skew,
            'kurtosis': kurt,
            'lambda': lambda_param
        }
    
    # Original
    transformations['original'] = evaluate_transformation('original', y)
    
    # Log transformation (if all positive)
    if (y > 0).all():
        y_log = np.log1p(y)
        transformations['log'] = evaluate_transformation('log', y_log)
    
    # Square root
    y_min = y.min()
    y_sqrt = np.sqrt(y - y_min + 1)
    transformations['sqrt'] = evaluate_transformation('sqrt', y_sqrt)
    
    # Box-Cox (if all positive)
    if (y > 0).all():
        try:
            y_boxcox, lambda_bc = boxcox(y)
            transformations['boxcox'] = evaluate_transformation('boxcox', y_boxcox, lambda_bc)
        except:
            print("Box-Cox transformation failed")
    
    # Yeo-Johnson
    try:
        y_yj, lambda_yj = yeojohnson(y)
        transformations['yeojohnson'] = evaluate_transformation('yeojohnson', y_yj, lambda_yj)
    except:
        print("Yeo-Johnson transformation failed")
    
    # Power transformer
    try:
        pt = PowerTransformer(method='yeo-johnson', standardize=False)
        y_pt = pt.fit_transform(y.values.reshape(-1, 1)).flatten()
        transformations['power_transformer'] = evaluate_transformation('power_transformer', y_pt)
    except:
        print("PowerTransformer failed")
    
    # Quantile transformer
    try:
        qt = QuantileTransformer(output_distribution='normal', random_state=42)
        y_qt = qt.fit_transform(y.values.reshape(-1, 1)).flatten()
        transformations['quantile_normal'] = evaluate_transformation('quantile_normal', y_qt)
    except:
        print("QuantileTransformer failed")
    
    # Select best transformation
    best_transform = max(transformations.keys(), key=lambda k: transformations[k]['score'])
    best_info = transformations[best_transform]
    
    print(f"🏆 Best transformation: {best_transform}")
    print(f"📊 Score: {best_info['score']:.4f}")
    print(f"📈 Normality p-value: {best_info['normality_p']:.4f}")
    print(f"📉 Skewness: {best_info['skewness']:.4f}")
    
    return best_transform, transformations

def apply_transformation(y, transform_method, transform_params=None):
    """Apply the selected transformation"""
    if transform_method == 'log':
        return np.log1p(y)
    elif transform_method == 'sqrt':
        return np.sqrt(y - y.min() + 1)
    elif transform_method == 'boxcox':
        return boxcox(y)[0]
    elif transform_method == 'yeojohnson':
        return yeojohnson(y)[0]
    elif transform_method == 'power_transformer':
        pt = PowerTransformer(method='yeo-johnson', standardize=False)
        return pt.fit_transform(y.values.reshape(-1, 1)).flatten()
    elif transform_method == 'quantile_normal':
        qt = QuantileTransformer(output_distribution='normal', random_state=42)
        return qt.fit_transform(y.values.reshape(-1, 1)).flatten()
    else:
        return y.copy()

def inverse_transform(y_transformed, transform_method, original_y, transform_params=None):
    """Apply inverse transformation"""
    if transform_method == 'log':
        return np.expm1(y_transformed)
    elif transform_method == 'sqrt':
        return (y_transformed ** 2) + original_y.min() - 1
    elif transform_method == 'boxcox':
        # Need to store lambda parameter
        return np.power(y_transformed * transform_params.get('lambda', 1) + 1, 
                       1 / transform_params.get('lambda', 1))
    # Add other inverse transformations as needed
    else:
        return y_transformed

# =====================================
# ROBUST MODEL IMPLEMENTATIONS
# =====================================

def create_robust_models():
    """Create a suite of robust models optimized for skewed data"""
    
    models = {}
    
    # 1. CatBoost with MAE Loss (Research-backed for skewed data)
    models['catboost_mae'] = cb.CatBoostRegressor(
        loss_function='MAE',
        iterations=500,
        depth=6,
        learning_rate=0.05,
        l2_leaf_reg=3.0,
        bootstrap_type='Bayesian',
        bagging_temperature=1.0,
        od_type='IncToDec',
        od_wait=50,
        random_state=42,
        verbose=False
    )
    
    # 2. CatBoost with Quantile Loss
    models['catboost_quantile'] = cb.CatBoostRegressor(
        loss_function='Quantile:alpha=0.5',
        iterations=500,
        depth=6,
        learning_rate=0.05,
        l2_leaf_reg=3.0,
        bootstrap_type='Bayesian',
        random_state=42,
        verbose=False
    )
    
    # 3. Quantile Regression (Median)
    models['quantile_median'] = QuantileRegressor(
        quantile=0.5,
        alpha=0.01,
        fit_intercept=True
    )
    
    # 4. Huber Regression (Robust to outliers)
    models['huber'] = HuberRegressor(
        epsilon=1.35,  # Standard robust parameter
        max_iter=1000,
        alpha=0.01
    )
    
    # 5. Theil-Sen Regressor (High breakdown point)
    models['theil_sen'] = TheilSenRegressor(
        random_state=42,
        max_subpopulation=1000
    )
    
    # 6. LightGBM with MAE
    models['lgb_mae'] = lgb.LGBMRegressor(
        objective='mae',
        n_estimators=500,
        learning_rate=0.05,
        num_leaves=31,
        feature_fraction=0.8,
        bagging_fraction=0.8,
        bagging_freq=5,
        random_state=42,
        verbose=-1
    )
    
    # 7. XGBoost with MAE
    models['xgb_mae'] = xgb.XGBRegressor(
        objective='reg:absoluteerror',
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    
    # 8. Random Forest (Extra robust)
    models['rf_robust'] = RandomForestRegressor(
        n_estimators=300,
        max_depth=15,
        min_samples_split=10,
        min_samples_leaf=5,
        max_features='sqrt',
        bootstrap=True,
        random_state=42,
        n_jobs=-1
    )
    
    return models

# =====================================
# ADVANCED HYPERPARAMETER OPTIMIZATION
# =====================================

def optimize_model_hyperparameters(model_name, X_train, y_train, cv_folds=3, n_trials=100):
    """Bayesian optimization for each model type"""
    
    def objective(trial):
        if model_name == 'catboost_mae':
            params = {
                'iterations': trial.suggest_int('iterations', 200, 800),
                'depth': trial.suggest_int('depth', 4, 10),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
                'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 2.0),
                'loss_function': 'MAE',
                'bootstrap_type': 'Bayesian',
                'random_state': 42,
                'verbose': False
            }
            model = cb.CatBoostRegressor(**params)
            
        elif model_name == 'lgb_mae':
            params = {
                'objective': 'mae',
                'n_estimators': trial.suggest_int('n_estimators', 200, 1000),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
                'num_leaves': trial.suggest_int('num_leaves', 20, 150),
                'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
                'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
                'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
                'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
                'random_state': 42,
                'verbose': -1
            }
            model = lgb.LGBMRegressor(**params)
            
        elif model_name == 'xgb_mae':
            params = {
                'objective': 'reg:absoluteerror',
                'n_estimators': trial.suggest_int('n_estimators', 200, 1000),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
                'max_depth': trial.suggest_int('max_depth', 3, 12),
                'subsample': trial.suggest_float('subsample', 0.6, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 2.0),
                'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 2.0),
                'random_state': 42
            }
            model = xgb.XGBRegressor(**params)
            
        elif model_name == 'quantile_median':
            params = {
                'quantile': 0.5,
                'alpha': trial.suggest_float('alpha', 0.001, 1.0, log=True),
                'fit_intercept': True
            }
            model = QuantileRegressor(**params)
            
        else:
            return float('inf')
        
        # Cross-validation
        cv_scores = cross_val_score(
            model, X_train, y_train, 
            cv=cv_folds, 
            scoring='neg_mean_absolute_error',  # MAE for robust evaluation
            n_jobs=-1
        )
        
        return -cv_scores.mean()
    
    # Run optimization
    study = optuna.create_study(
        direction='minimize',
        sampler=TPESampler(seed=42),
        pruner=MedianPruner(n_startup_trials=10, n_warmup_steps=5)
    )
    
    study.optimize(objective, n_trials=n_trials, timeout=600)  # 10 minutes max
    
    return study.best_params, study.best_value

# =====================================
# SHAP-BASED FEATURE SELECTION
# =====================================

def shap_feature_selection(models, X_train, y_train, X_val, top_k=None):
    """Use SHAP values for intelligent feature selection"""
    
    if not SHAP_AVAILABLE:
        print("⚠️ SHAP not available, skipping SHAP-based feature selection")
        return list(X_train.columns)
    
    print("🧠 SHAP-based feature selection...")
    
    feature_importance_scores = {}
    
    for model_name, model in models.items():
        print(f"  Analyzing {model_name}...")
        
        try:
            # Fit model
            model.fit(X_train, y_train)
            
            # Create SHAP explainer
            if hasattr(model, 'predict') and 'tree' in str(type(model)).lower():
                explainer = shap.TreeExplainer(model)
            else:
                explainer = shap.Explainer(model, X_train[:100])  # Sample for speed
            
            # Calculate SHAP values
            shap_values = explainer.shap_values(X_val[:500])  # Limit for speed
            
            # Get feature importance
            if isinstance(shap_values, list):
                shap_values = shap_values[0]
            
            feature_importance = np.abs(shap_values).mean(0)
            
            for i, feature in enumerate(X_train.columns):
                if feature not in feature_importance_scores:
                    feature_importance_scores[feature] = []
                feature_importance_scores[feature].append(feature_importance[i])
                
        except Exception as e:
            print(f"    SHAP analysis failed for {model_name}: {e}")
    
    # Average importance across models
    avg_importance = {}
    for feature, scores in feature_importance_scores.items():
        avg_importance[feature] = np.mean(scores)
    
    # Sort features by importance
    sorted_features = sorted(avg_importance.keys(), 
                           key=lambda x: avg_importance[x], 
                           reverse=True)
    
    # Select top features
    if top_k is None:
        top_k = max(20, len(sorted_features) // 3)  # At least 20 or 1/3 of features
    
    selected_features = sorted_features[:top_k]
    
    print(f"✅ Selected {len(selected_features)} features out of {len(sorted_features)}")
    print(f"🔝 Top 10 features: {selected_features[:10]}")
    
    return selected_features

# =====================================
# MAIN PIPELINE
# =====================================

def main():
    print("🚀 Starting Ultimate Pollution Prediction Pipeline...")
    
    # Load data
    print("📂 Loading data...")
    train_data = pd.read_csv('train.csv')
    test_data = pd.read_csv('test.csv')
    
    print(f"📊 Train data shape: {train_data.shape}")
    print(f"📊 Test data shape: {test_data.shape}")
    
    # Identify target column
    target_col = None
    for col in ['target', 'pollution_value']:
        if col in train_data.columns:
            target_col = col
            break
    
    if target_col is None:
        raise ValueError("Target column not found!")
    
    # Feature engineering
    print("\\n🔧 Ultimate feature engineering...")
    train_enhanced = create_ultimate_features(train_data, is_train=True)
    test_enhanced = create_ultimate_features(test_data, is_train=False)
    
    # Align features
    train_features = set(train_enhanced.columns) - {target_col, 'id'}
    test_features = set(test_enhanced.columns) - {'id'}
    common_features = sorted(train_features.intersection(test_features))
    
    X = train_enhanced[common_features].copy()
    y = train_enhanced[target_col].copy()
    X_test = test_enhanced[common_features].copy()
    
    print(f"✅ Final feature count: {len(common_features)}")
    
    # Target transformation optimization
    print("\\n🎯 Optimizing target transformation...")
    best_transform, all_transforms = optimize_target_transformation(y)
    y_transformed = apply_transformation(y, best_transform)
    
    # Train-validation split
    print("\\n📊 Creating robust train-validation split...")
    
    # Stratified split based on target quantiles
    try:
        y_bins = pd.qcut(y_transformed, q=5, labels=False, duplicates='drop')
        stratify = y_bins
    except:
        stratify = None
    
    X_train, X_val, y_train, y_val = train_test_split(
        X, y_transformed,
        test_size=0.2,
        random_state=42,
        stratify=stratify
    )
    
    # Multiple scaling strategies
    print("\\n⚖️ Applying multiple scaling strategies...")
    
    scalers = {
        'robust': RobustScaler(),
        'standard': StandardScaler(),
        'minmax': MinMaxScaler()
    }
    
    scaled_data = {}
    for name, scaler in scalers.items():
        scaled_data[name] = {
            'X_train': scaler.fit_transform(X_train),
            'X_val': scaler.transform(X_val),
            'X_test': scaler.transform(X_test),
            'scaler': scaler
        }
    
    # Create robust models
    print("\\n🤖 Creating robust model suite...")
    base_models = create_robust_models()
    
    # Quick evaluation to select best scaling
    print("\\n🔍 Selecting optimal scaling method...")
    
    best_scaler = 'robust'
    best_score = float('inf')
    
    for scaler_name, data in scaled_data.items():
        print(f"  Testing {scaler_name} scaling...")
        
        # Quick test with CatBoost
        model = cb.CatBoostRegressor(
            loss_function='MAE', iterations=100, verbose=False, random_state=42
        )
        model.fit(data['X_train'], y_train)
        pred = model.predict(data['X_val'])
        score = mean_absolute_error(y_val, pred)
        
        print(f"    MAE: {score:.6f}")
        
        if score < best_score:
            best_score = score
            best_scaler = scaler_name
    
    print(f"🏆 Best scaling method: {best_scaler}")
    
    # Use best scaling
    X_train_scaled = scaled_data[best_scaler]['X_train']
    X_val_scaled = scaled_data[best_scaler]['X_val']
    X_test_scaled = scaled_data[best_scaler]['X_test']
    final_scaler = scaled_data[best_scaler]['scaler']
    
    # SHAP-based feature selection
    print("\\n🧠 SHAP-based feature selection...")
    
    # Train subset of models for feature selection
    selection_models = {
        'catboost': base_models['catboost_mae'],
        'lgb': base_models['lgb_mae']
    }
    
    selected_features = shap_feature_selection(
        selection_models, 
        pd.DataFrame(X_train_scaled, columns=common_features),
        y_train,
        pd.DataFrame(X_val_scaled, columns=common_features),
        top_k=min(50, len(common_features))
    )
    
    # Apply feature selection
    feature_indices = [common_features.index(f) for f in selected_features if f in common_features]
    X_train_selected = X_train_scaled[:, feature_indices]
    X_val_selected = X_val_scaled[:, feature_indices]
    X_test_selected = X_test_scaled[:, feature_indices]
    
    print(f"✅ Using {len(selected_features)} selected features")
    
    # Hyperparameter optimization for key models
    print("\\n⚙️ Bayesian hyperparameter optimization...")
    
    optimized_models = {}
    optimization_results = {}
    
    key_models = ['catboost_mae', 'lgb_mae', 'xgb_mae', 'quantile_median']
    
    for model_name in key_models:
        print(f"\\n  Optimizing {model_name}...")
        
        try:
            best_params, best_score = optimize_model_hyperparameters(
                model_name, X_train_selected, y_train, cv_folds=3, n_trials=50
            )
            
            print(f"    Best MAE: {best_score:.6f}")
            
            # Create optimized model
            if model_name == 'catboost_mae':
                optimized_models[model_name] = cb.CatBoostRegressor(**best_params)
            elif model_name == 'lgb_mae':
                optimized_models[model_name] = lgb.LGBMRegressor(**best_params)
            elif model_name == 'xgb_mae':
                optimized_models[model_name] = xgb.XGBRegressor(**best_params)
            elif model_name == 'quantile_median':
                optimized_models[model_name] = QuantileRegressor(**best_params)
            
            optimization_results[model_name] = {
                'best_params': best_params,
                'best_score': best_score
            }
            
        except Exception as e:
            print(f"    Optimization failed: {e}")
            # Use default model
            optimized_models[model_name] = base_models[model_name]
    
    # Add other robust models
    optimized_models['huber'] = base_models['huber']
    optimized_models['theil_sen'] = base_models['theil_sen']
    
    # Train all models and evaluate
    print("\\n🎯 Training and evaluating all models...")
    
    model_results = {}
    val_predictions = {}
    test_predictions = {}
    
    for model_name, model in optimized_models.items():
        print(f"  Training {model_name}...")
        
        try:
            # Train
            model.fit(X_train_selected, y_train)
            
            # Predict
            val_pred = model.predict(X_val_selected)
            test_pred = model.predict(X_test_selected)
            
            # Evaluate
            val_mae = mean_absolute_error(y_val, val_pred)
            val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))
            val_r2 = r2_score(y_val, val_pred)
            
            model_results[model_name] = {
                'mae': val_mae,
                'rmse': val_rmse,
                'r2': val_r2
            }
            
            val_predictions[model_name] = val_pred
            test_predictions[model_name] = test_pred
            
            print(f"    MAE: {val_mae:.6f}, RMSE: {val_rmse:.6f}, R²: {val_r2:.6f}")
            
        except Exception as e:
            print(f"    Training failed: {e}")
    
    # Multi-level stacking ensemble
    print("\\n🏗️ Building multi-level stacking ensemble...")
    
    # Level 1: Base models
    level1_models = []
    for model_name, model in optimized_models.items():
        if model_name in val_predictions:  # Only successful models
            level1_models.append((model_name, model))
    
    # Level 2: Meta-learners
    meta_learners = {
        'ridge_robust': Ridge(alpha=10.0),
        'huber_meta': HuberRegressor(epsilon=1.35),
        'quantile_meta': QuantileRegressor(quantile=0.5, alpha=0.1)
    }
    
    stacking_results = {}
    
    for meta_name, meta_learner in meta_learners.items():
        print(f"  Testing meta-learner: {meta_name}")
        
        try:
            stacking_model = StackingRegressor(
                estimators=level1_models[:5],  # Top 5 models
                final_estimator=meta_learner,
                cv=3,
                n_jobs=-1
            )
            
            stacking_model.fit(X_train_selected, y_train)
            stacking_pred = stacking_model.predict(X_val_selected)
            stacking_test_pred = stacking_model.predict(X_test_selected)
            
            stacking_mae = mean_absolute_error(y_val, stacking_pred)
            stacking_rmse = np.sqrt(mean_squared_error(y_val, stacking_pred))
            stacking_r2 = r2_score(y_val, stacking_pred)
            
            stacking_results[meta_name] = {
                'model': stacking_model,
                'mae': stacking_mae,
                'rmse': stacking_rmse,
                'r2': stacking_r2,
                'val_pred': stacking_pred,
                'test_pred': stacking_test_pred
            }
            
            print(f"    MAE: {stacking_mae:.6f}, RMSE: {stacking_rmse:.6f}")
            
        except Exception as e:
            print(f"    Stacking failed: {e}")
    
    # Quantile ensemble (25th, 50th, 75th percentiles)
    print("\\n📊 Building quantile ensemble...")
    
    try:
        quantile_models = {}
        quantile_predictions = {}
        
        for q in [0.25, 0.5, 0.75]:
            q_model = QuantileRegressor(quantile=q, alpha=0.01)
            q_model.fit(X_train_selected, y_train)
            
            q_val_pred = q_model.predict(X_val_selected)
            q_test_pred = q_model.predict(X_test_selected)
            
            quantile_models[f'q{int(q*100)}'] = q_model
            quantile_predictions[f'q{int(q*100)}'] = {
                'val': q_val_pred,
                'test': q_test_pred
            }
        
        # Average of quantiles (robust prediction)
        quantile_val_pred = np.mean([
            quantile_predictions['q25']['val'],
            quantile_predictions['q50']['val'],
            quantile_predictions['q75']['val']
        ], axis=0)
        
        quantile_test_pred = np.mean([
            quantile_predictions['q25']['test'],
            quantile_predictions['q50']['test'],
            quantile_predictions['q75']['test']
        ], axis=0)
        
        quantile_mae = mean_absolute_error(y_val, quantile_val_pred)
        quantile_rmse = np.sqrt(mean_squared_error(y_val, quantile_val_pred))
        
        print(f"✅ Quantile ensemble - MAE: {quantile_mae:.6f}, RMSE: {quantile_rmse:.6f}")
        
    except Exception as e:
        print(f"Quantile ensemble failed: {e}")
        quantile_val_pred = None
        quantile_test_pred = None
    
    # Select final model
    print("\\n🏆 Selecting final model...")
    
    all_results = {}
    
    # Individual models
    for name, results in model_results.items():
        all_results[name] = results
    
    # Stacking models
    for name, results in stacking_results.items():
        all_results[f'stacking_{name}'] = {
            'mae': results['mae'],
            'rmse': results['rmse'],
            'r2': results['r2']
        }
    
    # Quantile ensemble
    if quantile_val_pred is not None:
        all_results['quantile_ensemble'] = {
            'mae': quantile_mae,
            'rmse': quantile_rmse,
            'r2': r2_score(y_val, quantile_val_pred)
        }
    
    # Find best model by MAE (robust metric)
    best_model_name = min(all_results.keys(), key=lambda k: all_results[k]['mae'])
    best_mae = all_results[best_model_name]['mae']
    
    print(f"🥇 Best model: {best_model_name}")
    print(f"🎯 Best MAE: {best_mae:.6f}")
    
    # Get final predictions
    if best_model_name.startswith('stacking_'):
        meta_name = best_model_name.replace('stacking_', '')
        final_test_pred = stacking_results[meta_name]['test_pred']
    elif best_model_name == 'quantile_ensemble':
        final_test_pred = quantile_test_pred
    else:
        final_test_pred = test_predictions[best_model_name]
    
    # Apply inverse transformation
    print("\\n🔄 Applying inverse transformation...")
    
    final_test_pred_original = inverse_transform(
        final_test_pred, best_transform, y
    )
    
    # Create submission
    print("\\n📝 Creating final submission...")
    
    test_ids = test_data['id'] if 'id' in test_data.columns else range(len(test_data))
    
    submission = pd.DataFrame({
        'id': test_ids,
        'target': final_test_pred_original
    })
    
    # Save submission
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    submission_filename = f'submissions/ultimate_submission_{best_model_name}_{timestamp}.csv'
    submission.to_csv(submission_filename, index=False)
    
    # Save models and results
    results_summary = {
        'best_model': best_model_name,
        'best_mae': float(best_mae),
        'transform_method': best_transform,
        'scaling_method': best_scaler,
        'selected_features': selected_features,
        'all_results': {k: {kk: float(vv) for kk, vv in v.items()} 
                       for k, v in all_results.items()},
        'optimization_results': optimization_results
    }
    
    with open(f'results/ultimate_results_{timestamp}.json', 'w') as f:
        json.dump(results_summary, f, indent=2)
    
    # Save final model
    if best_model_name.startswith('stacking_'):
        meta_name = best_model_name.replace('stacking_', '')
        final_model = stacking_results[meta_name]['model']
        joblib.dump(final_model, f'models/ultimate_model_{timestamp}.pkl')
    elif best_model_name in optimized_models:
        joblib.dump(optimized_models[best_model_name], f'models/ultimate_model_{timestamp}.pkl')
    
    # Save preprocessing objects
    joblib.dump(final_scaler, f'models/ultimate_scaler_{timestamp}.pkl')
    joblib.dump(selected_features, f'models/ultimate_features_{timestamp}.pkl')
    
    print(f"\\n🎉 ULTIMATE PIPELINE COMPLETE!")
    print(f"📁 Submission saved: {submission_filename}")
    print(f"🏆 Expected improvement: 25-55% over baseline")
    print(f"📊 Current MAE: {best_mae:.6f}")
    print(f"🎯 Target achieved: Implementation of 10+ research-backed techniques")
    print("=" * 80)

if __name__ == "__main__":
    main()


🚀 ULTIMATE POLLUTION PREDICTION MODEL
📚 Based on 25+ Academic Papers Research
🎯 Target: 25-55% RMSE Improvement
🚀 Starting Ultimate Pollution Prediction Pipeline...
📂 Loading data...
📊 Train data shape: (7649, 8)
📊 Test data shape: (2739, 7)
\n🔧 Ultimate feature engineering...
🔧 Creating ultimate features... Initial shape: (7649, 8)
Pollution_value target column preserved for feature engineering
📊 Advanced missing value imputation...
Found 26 NaN values, filling with appropriate values...
Creating polynomial features from: ['latitude', 'longitude', 'hour']
Added 3 polynomial interaction features
🌍 Creating spatial clustering features...


  File "c:\Users\damod\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\Users\damod\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\damod\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\damod\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


⏰ Creating enhanced cyclical features...
📊 Creating spatial binning features...
📏 Creating distance-based features...
� Creating temporal pattern features...
🌤️ Creating weather proxy features...
� Creating interaction features...
📈 Creating statistical aggregation features...
🏷️ Advanced categorical encoding...
🛡️ Creating outlier-resistant features...
🧹 Final feature cleanup...
Cleaning 6 NaN values...
Target column restored after feature engineering
✅ Ultimate feature engineering completed!
📊 Final shape: (7649, 106)
🆕 Added 98 new features
🔍 Data quality: 0 NaN, 0 infinite values
🔧 Creating ultimate features... Initial shape: (2739, 7)
📊 Advanced missing value imputation...
Creating polynomial features from: ['latitude', 'longitude', 'hour']
Added 3 polynomial interaction features
🌍 Creating spatial clustering features...
⏰ Creating enhanced cyclical features...
📊 Creating spatial binning features...
📏 Creating distance-based features...
� Creating temporal pattern features...
🌤️ C

[I 2025-08-30 20:57:37,857] A new study created in memory with name: no-name-367129e6-4ef2-42f9-840a-7bbc7c41d160


✅ Selected 50 features out of 100
🔝 Top 10 features: ['distance_from_center', 'feature_mean', 'latitude_longitude_interaction', 'latitude_longitude_ratio', 'weather_proxy', 'dist_from_lon_min', 'longitude_winsorized', 'spatial_bin', 'distance_to_kmeans_10', 'poly_latitude longitude']
✅ Using 50 selected features
\n⚙️ Bayesian hyperparameter optimization...
\n  Optimizing catboost_mae...


...

    Best MAE: 0.139680
\n  Optimizing xgb_mae...


[I 2025-08-30 21:16:16,244] Trial 0 finished with value: 0.1456803364064417 and parameters: {'n_estimators': 500, 'learning_rate': 0.19063571821788408, 'max_depth': 10, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'reg_alpha': 0.3119890406724053, 'reg_lambda': 0.11616722433639892}. Best is trial 0 with value: 0.1456803364064417.
[I 2025-08-30 21:16:38,698] Trial 1 finished with value: 0.14757535558852017 and parameters: {'n_estimators': 893, 'learning_rate': 0.12421185223120967, 'max_depth': 10, 'subsample': 0.608233797718321, 'colsample_bytree': 0.9879639408647978, 'reg_alpha': 1.6648852816008435, 'reg_lambda': 0.4246782213565523}. Best is trial 0 with value: 0.1456803364064417.
[I 2025-08-30 21:16:40,778] Trial 2 finished with value: 0.14288865440340595 and parameters: {'n_estimators': 345, 'learning_rate': 0.044846856872152424, 'max_depth': 6, 'subsample': 0.8099025726528951, 'colsample_bytree': 0.7727780074568463, 'reg_alpha': 0.5824582803960838, 'reg_la

    Best MAE: 0.138376
\n  Optimizing quantile_median...


[I 2025-08-30 21:26:32,811] Trial 0 finished with value: 0.19157611004091993 and parameters: {'alpha': 0.013292918943162165}. Best is trial 0 with value: 0.19157611004091993.
[I 2025-08-30 21:26:33,460] Trial 1 finished with value: 0.2407087888293328 and parameters: {'alpha': 0.711447600934342}. Best is trial 0 with value: 0.19157611004091993.
[I 2025-08-30 21:26:34,160] Trial 2 finished with value: 0.24071958842823168 and parameters: {'alpha': 0.15702970884055384}. Best is trial 0 with value: 0.19157611004091993.
[I 2025-08-30 21:26:34,953] Trial 3 finished with value: 0.21993709786004537 and parameters: {'alpha': 0.06251373574521749}. Best is trial 0 with value: 0.19157611004091993.
[I 2025-08-30 21:26:36,992] Trial 4 finished with value: 0.18245027604896424 and parameters: {'alpha': 0.0029380279387035343}. Best is trial 4 with value: 0.18245027604896424.
[I 2025-08-30 21:26:38,764] Trial 5 finished with value: 0.18245109170638665 and parameters: {'alpha': 0.0029375384576328283}. Bes

    Best MAE: 0.180704
\n🎯 Training and evaluating all models...
  Training catboost_mae...
0:	learn: 0.2963359	total: 49.1ms	remaining: 19.2s
1:	learn: 0.2926234	total: 87.8ms	remaining: 17.1s
2:	learn: 0.2891522	total: 127ms	remaining: 16.4s
3:	learn: 0.2859323	total: 165ms	remaining: 15.9s
4:	learn: 0.2826636	total: 205ms	remaining: 15.8s
5:	learn: 0.2795666	total: 245ms	remaining: 15.7s
6:	learn: 0.2765936	total: 285ms	remaining: 15.6s
7:	learn: 0.2738215	total: 322ms	remaining: 15.4s
8:	learn: 0.2711008	total: 361ms	remaining: 15.3s
9:	learn: 0.2683987	total: 401ms	remaining: 15.3s
10:	learn: 0.2659392	total: 442ms	remaining: 15.3s
11:	learn: 0.2635245	total: 479ms	remaining: 15.1s
12:	learn: 0.2611116	total: 517ms	remaining: 15s
13:	learn: 0.2589013	total: 554ms	remaining: 14.9s
14:	learn: 0.2567804	total: 591ms	remaining: 14.8s
15:	learn: 0.2547658	total: 630ms	remaining: 14.8s
16:	learn: 0.2526604	total: 671ms	remaining: 14.8s
17:	learn: 0.2507404	total: 708ms	remaining: 14.7s
